#  THIS IS ALL OUT OF DATE!!!! I'VE MADE A LOT OF IMPROVEMENTS, STILL TINKERING, NEED TO WRITE IT UP PROPERLY SOON...



BAD FLATMAPS

Easy (and hacky!!) way to import freesurfer ROIs into pycortex

BEFORE YOU DO THIS MAKE SURE TO BACK UP ANY KIND OF PYCORTEX STUFF YOU DON'T WANT OVERWRITTEN!!!

Why would you do this? Because pycortex is beautiful, but you are lazy... 

It is really nice to be able to toggle the ROIs on and off in pycortex viewer; but to do this you have to have them defined on the svg. To define them on the SVG, you need to draw them in inkscape. But maybe you already used freesurfer, and don't want to... whatever

This helps with that...

It also works with anything defined as a boolean array. 

Check out an example at:




In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
opj = os.path.join

In [ ]:
sub = 'sub-01'                                      # Subject number
fs_dir = '~/home/project1/derivatives/freesurfer'   # e.g., /home/project1/derivatives/freesurfer
ctx_dir = '~/home/project1/derivatives/pycortex' # e.g., /home/project1/derivatives/freesurfer/label



In [ ]:
from dag_prf_utils.pycortex import set_ctx_path,PyctxMaker

# Make sure pycortex is pointing to correct folder
# set_ctx_path('/path/to/pycortex/files')

pm = PyctxMaker(
    sub = sub,
    fs_dir = fs_dir, 
    ctx_path='/path/to/pycortex/files'
    )


## Bad flatmaps
Already got flatmaps? then skip this stage

Pycortex needs flatmaps. In the form of flat_*h.gii in the pycortex folder for that subject. If you are very lazy (like me) and don't want to go through the trouble of flattening - i offer a DODGY way  to flatten the surface. In brief I:

[1] Take the coordinates for the vertices from "sphere" surface (i.e., inflated to a sphere) 

[2] Treat this brain sphere like a globe; and transform the coordinates to 2D, using the "latitude" and "longitude" 

[3] If specificied I will centre the latitude and longitude around a group of points (i.e., to put your ROI to the centre). You can also choose to "chop out" vertices which are not near this group of points you want to centre on

[4] Find the faces which are now stretched and cut them... Thats it basically

I'm working on ways of making this smoother; but ultimately if you care about the flatmap representation (and you aren't just doing this to get the ROIs into pycortex) - then you should probably take the time to do it properly with freesurfer/blender

You can do that all in one step using the code below...

In [ ]:
pm.make_flat_map_LATLONG()

But lets walk through it step by step

[1] Take the coordinates from the sphere...

In [ ]:
# Read in the mesh info for the path to  the "sphere files "
def dag_read_fs_mesh(filepath, return_xyz=False):
    """Adapted from https://github.com/nipy/nibabel/blob/master/nibabel/freesurfer/io.py
    ...
    Read a triangular format Freesurfer surface mesh.

    Parameters
    ----------
    filepath : str
        Path to surface file.

    Returns
    -------
    coords : numpy array
        nvtx x 3 array of vertex (x, y, z) coordinates.
    faces : numpy array
        nfaces x 3 array of defining mesh triangles.
    """

    TRIANGLE_MAGIC = 16777214
    with open(filepath, 'rb') as fobj:

        magic = dag_fread3(fobj)
        create_stamp = fobj.readline().rstrip(b'\n').decode('utf-8')
        fobj.readline()
        vnum = np.fromfile(fobj, '>i4', 1)[0]
        fnum = np.fromfile(fobj, '>i4', 1)[0]
        coords = np.fromfile(fobj, '>f4', vnum * 3).reshape(vnum, 3)
        faces = np.fromfile(fobj, '>i4', fnum * 3).reshape(fnum, 3)

    coords = coords.astype(np.float64)  # XXX: due to mayavi bug on mac 32bits

    mesh_info = {
        'vnum' : vnum,
        'fnum' : fnum,
        'coords' : coords,
        'faces' : faces,        
    }
    if return_xyz:
        new_mesh_info = {}
        new_mesh_info['coords'] = mesh_info['coords']
        new_mesh_info['faces'] = mesh_info['faces']                                  
        new_mesh_info['x']= mesh_info['coords'][:,0]
        new_mesh_info['y']= mesh_info['coords'][:,1]
        new_mesh_info['z']= mesh_info['coords'][:,2]
        new_mesh_info['i']= mesh_info['faces'][:,0]
        new_mesh_info['j']= mesh_info['faces'][:,1]
        new_mesh_info['k']= mesh_info['faces'][:,2]        
        mesh_info = new_mesh_info

    return mesh_info

def dag_fread3(fobj): # Copied from nibabel
    """Read a 3-byte int from an open binary file object

    Parameters
    ----------
    fobj : file
        File descriptor

    Returns
    -------
    n : int
        A 3 byte int
    """
    b1, b2, b3 = np.fromfile(fobj, '>u1', 3)
    return (b1 << 16) + (b2 << 8) + b3
# Lets load the inflated and sphere meshes
mesh_info = {}
for mesh in ['inflated','sphere']:
    mesh_info[mesh] = {}
    for hemi in ['lh','rh']:
        mesh_info[mesh][hemi] = dag_read_fs_mesh(
            os.path.join(fs_dir,sub,'surf',hemi+'.'+mesh), return_xyz=True)
# The number of vertices in each hemisphere will be useful to know...
nvx = {}
for hemi in ['lh','rh']:
    nvx[hemi] = mesh_info['sphere'][hemi]['x'].shape[0]


[2] Flatten and cut to wherever you want

In [ ]:
# Where do we want the centre of the flat map to be? 
# And which vertices do we want to include in the flat map?
# - we could set this based on ROIs
vx_of_interest = {}
for hemi in ['lh', 'rh']:
    # Including all of them here... add your own mask
    vx_of_interest[hemi] = np.ones(nvx[hemi], dtype=bool)

# Now lets find the box surrounding these pts


def dag_cut_box(mesh_info, **kwargs):
    '''Find vx to cut for a box

    '''
    border_buffer = kwargs.get('border_buffer', 10) # % of the max distance

    borders = {}
    borders['x_min'] = kwargs.get('x_min', None)
    borders['x_max'] = kwargs.get('x_max', None)
    borders['y_min'] = kwargs.get('y_min', None)
    borders['y_max'] = kwargs.get('y_max', None)
    borders['z_min'] = kwargs.get('z_min', None)
    borders['z_max'] = kwargs.get('z_max', None)
    vx_bool = kwargs.get('vx_bool', None)
    if vx_bool is not None:
        for b in ['x', 'y', 'z']:
            borders[f'{b}_min'] = mesh_info[b][vx_bool].min()
            borders[f'{b}_max'] = mesh_info[b][vx_bool].max()
    # Now find the vx to include 
    vx_to_include = np.ones_like(mesh_info['x'], dtype=bool)
    # find the biggest distance
    abs_border_buffer = 0
    for b in ['x', 'y', 'z']:
        abs_border_buffer = max(
            abs_border_buffer, 
            (mesh_info[b].max() - mesh_info[b].min()) * border_buffer / 100)

    for b in ['x', 'y', 'z']:
        if borders[f'{b}_min'] is not None:
            vx_to_include &= mesh_info[b]>borders[f'{b}_min'] - abs_border_buffer
        if borders[f'{b}_max'] is not None:
            vx_to_include &= mesh_info[b]<borders[f'{b}_max'] + abs_border_buffer

    return vx_to_include
vx_in_cut_box = {}
for hemi in ['lh', 'rh']:
    vx_in_cut_box[hemi] = dag_cut_box(mesh_info['inflated'][hemi], vx_bool=vx_of_interest[hemi])


Now lets get the flat coordinates...

In [ ]:
def dag_sph2flat(coords, **kwargs):
    '''Flatten a sphere to 2D
    This is a bad way to flatten the cortex
    You should probably do proper surface cuts etc...
    But this is a quick and dirty way to do it. And may even be ok when you do if for ROIs...
    It is a work in progress, which may improve overtime...
    
    TODO: 
    * option to define the centre... 
    * better way to do the projection?
    https://en.wikipedia.org/wiki/Map_projection
    
    
    # Adjust longitudes based on the new center longitude
    lon -= center_lon
    
    # Ensure lon is within the range [-pi, pi]
    lon = (lon + np.pi) % (2 * np.pi) - np.pi    
    '''
    # First move to 0,0,0...
    coords -= coords.mean(axis=0)
    # Sanity check -> is distance to 0 should be the same for all points    
    d20 = np.sqrt(np.sum(coords**2, axis=1))
    atol = 10
    if not np.allclose(d20, d20[0], atol=atol):
        print(f'Warning: Not all points are equidistant from 0,0,0 atol={atol}')

    # Now flatten to 2D using longitude and latitude
    x,y,z = coords[:,0], coords[:,1], coords[:,2]
    lat= np.arctan2(z, np.sqrt(x**2 + y**2)) * 2 # 
    lon = np.arctan2(y, x)
    # print(f'Lat: {lat.min()} {lat.max()}')
    # print(f'Lon: {lon.min()} {lon.max()}')


    # Adjust longitudes based on the new center longitude
    centre_bool = kwargs.get('centre_bool', None)
    if centre_bool is not None:
        centre_lat = lat[centre_bool].mean()
        centre_lon = lon[centre_bool].mean()    
        print('centering!')
        lat -= centre_lat
        lat = (lat + np.pi) % (2 * np.pi) - np.pi
        lon -= centre_lon
        lon = (lon + np.pi) % (2 * np.pi) - np.pi

    return lon, lat

def dag_bad_flatten(sphere_mesh_info, **kwargs):
    '''Take the spherical coordinates
    This is a bad way to flatten the sphere - you should probably do proper surface cuts etc...    
    flatten them to 2D (just polar)
    '''
    z = kwargs.get('z', 0)
    bad_flat = {}
    p1, p2 = dag_sph2flat(sphere_mesh_info['coords'], **kwargs)
    # find relative scale...
    mag = sphere_mesh_info['coords'].max() / p1.max() 
    bad_flat['x'] = p1 * mag
    bad_flat['y'] = p2 * mag
    bad_flat['z'] = np.ones_like(bad_flat['x']) * z
    
    # Cut faces with any of the "cut_bool" vertices in them
    cut_bool = kwargs.get('cut_bool', None)
    if cut_bool is not None:
        # Find any faces with vertices in the cut
        cut_vx = np.where(cut_bool)[0]
        cut_faces = np.isin(sphere_mesh_info['i'], cut_vx) + np.isin(sphere_mesh_info['j'], cut_vx) + np.isin(sphere_mesh_info['k'], cut_vx)
        cut_faces = cut_faces>0
        print(f'Cutting {cut_faces.sum()} faces out of {cut_faces.shape[0]}')
    else:
        cut_faces = np.zeros(sphere_mesh_info['i'].shape[0], dtype=bool)
    
    
    # Find the mean length of an edge 
    face_lengths = []
    for i_f in range(sphere_mesh_info['i'].shape[0]):
        ei2j = np.sqrt(
            (bad_flat['x'][sphere_mesh_info['i'][i_f]] - bad_flat['x'][sphere_mesh_info['j'][i_f]])**2 +
            (bad_flat['y'][sphere_mesh_info['i'][i_f]] - bad_flat['y'][sphere_mesh_info['j'][i_f]])**2
        )
        ei2k = np.sqrt(
            (bad_flat['x'][sphere_mesh_info['i'][i_f]] - bad_flat['x'][sphere_mesh_info['k'][i_f]])**2 +
            (bad_flat['y'][sphere_mesh_info['i'][i_f]] - bad_flat['y'][sphere_mesh_info['k'][i_f]])**2
        )
        ej2k = np.sqrt(
            (bad_flat['x'][sphere_mesh_info['j'][i_f]] - bad_flat['x'][sphere_mesh_info['k'][i_f]])**2 +
            (bad_flat['y'][sphere_mesh_info['j'][i_f]] - bad_flat['y'][sphere_mesh_info['k'][i_f]])**2
        )
        face_lengths.append(ei2j+ei2k+ej2k)
    face_lengths = np.array(face_lengths)
    m_face_lengths = face_lengths.mean()
    std_face_lengths = face_lengths.std()
    # Find the faces with edges > 4*std
    f_w_long_edges = face_lengths > m_face_lengths + 4*std_face_lengths
    
    cut_faces |= f_w_long_edges
    print(f'Faces with long edges: {f_w_long_edges.sum()}')    

    bad_flat['faces']  = sphere_mesh_info['faces'][~cut_faces,:]
    bad_flat['i']      = sphere_mesh_info['i'][~cut_faces]
    bad_flat['j']      = sphere_mesh_info['j'][~cut_faces]
    bad_flat['k']      = sphere_mesh_info['k'][~cut_faces]

    pts = np.vstack([bad_flat['x'],bad_flat['y'], bad_flat['z']]).T    
    pts[cut_bool] = 0 # Move pts to cut to 0
    polys = bad_flat['faces']
    return pts, polys
for hemi in ['lh', 'rh']:
    pts, polys = dag_bad_flatten(
        mesh_info['sphere'][hemi], 
        cut_bool=vx_in_cut_box[hemi],       # Which vertices to include... (more inclusive than centre bool to try and avoid dangling bits) 
        centre_bool=vx_of_interest[hemi]    # Where to centre 2D x,y (latitude and lognitude)
        )
    # Do some cleaning with pycortex software...
    import cortex
    polys = cortex.freesurfer._remove_disconnected_polys(polys)
    flat = cortex.freesurfer._move_disconnect_points_to_zero(flat, polys)
    # Save the flat map as a .gii file in the subjects pycortex folder
    flat_surf_path = opj('path/to/sub/ctx/file', 'surfaces', f'flat_{hemi}.gii')
    print("saving to %s"%flat_surf_path)
    cortex.formats.write_gii(flat_surf_path, pts=flat, polys=polys)

# Pycortex should detect it, and import it when if you run quickshow
random_surface = cortex.Vertex(
    data=np.random.randn(nvx['lh']+nvx['rh']), 
    subject=sub,
    )
cortex.quickshow(
    random_surface
)

## Add the ROIs (from freesurfer)
To add ROIs we 

[1] Load the boolean array from freesurfer

[2] Find the vertices on the edge of ROIs

[3] Find the order for these vertices 

[4] Draw them onto our SVG file


Again I've written this so you can do it just using this line of code

In [ ]:
roi_list = ['V1', 'V2', 'V3'] # roi names in you're freesurfer label folder
pm.add_rois_to_svg(roi_list)

Again we can go through step by step...


In [ ]:
# [1] Say we have an boolean ROI for V1
v1_bool = np.ones(nvx['lh']+nvx['rh'], dtype=bool) # Replace with actual ROI 

# [2] Find Edge vertices, in order (to draw a closed loop)
# Use these 4 functions...
def dag_find_border_vx_in_order(roi_bool, mesh_info, return_coords=False):
    '''dag_find_border_vx_in_order
    Find the border vertices in order to draw a closed loop    
    '''
    outer_edge_list = dag_get_roi_border_edge(roi_bool, mesh_info)    
    border_vx = dag_order_edges(outer_edge_list)
    if not return_coords:
        return border_vx
    # border_vx = sum(border_vx, []) # flatten list
    border_vx_coords = []
    for i_vx in border_vx:
        border_vx_coords.append([
            mesh_info['x'][i_vx],
            mesh_info['y'][i_vx],
            mesh_info['z'][i_vx],
        ])

    return border_vx,border_vx_coords


def dag_get_roi_border_edge(roi_bool, mesh_info):
    '''
    Find those vx which are on a border... 
    '''
    roi_idx = np.where(roi_bool)[0]
    in_face_x = {}
    for face_x in ['i', 'j', 'k']:
        in_face_x[face_x] = np.isin(mesh_info[face_x], roi_idx) * 1.0
    f_w_outer_edge = (in_face_x['i'] + in_face_x['j'] + in_face_x['k']) == 2
    f_w_outer_edge = np.where(f_w_outer_edge)[0]
    ij_faces_match = (in_face_x['i'][f_w_outer_edge] + in_face_x['j'][f_w_outer_edge])==2
    jk_faces_match = (in_face_x['j'][f_w_outer_edge] + in_face_x['k'][f_w_outer_edge])==2
    ki_faces_match = (in_face_x['k'][f_w_outer_edge] + in_face_x['i'][f_w_outer_edge])==2

    ij_faces_match = f_w_outer_edge[ij_faces_match]
    jk_faces_match = f_w_outer_edge[jk_faces_match]
    ki_faces_match = f_w_outer_edge[ki_faces_match]
    
    outer_edge_list = []
    # ij
    outer_edge_list.append(
        np.vstack([mesh_info['i'][ij_faces_match],mesh_info['j'][ij_faces_match]]),
    )
    # jk
    outer_edge_list.append(
        np.vstack([mesh_info['j'][jk_faces_match],mesh_info['k'][jk_faces_match]]),
    )
    # ki     
    outer_edge_list.append(
        np.vstack([mesh_info['k'][ki_faces_match],mesh_info['i'][ki_faces_match]]),
    )    
    # for face in ij_faces_match:
    #     outer_edge_list.append([
    #         mesh_info['i'][face]
    #     ])
    outer_edge_list = np.hstack(outer_edge_list).T
    return outer_edge_list

def dag_order_edges(edges):
    '''Order the edges to form a closed loop'''
    unique_vx = list(np.unique(edges.flatten()))
    # print(unique_vx[0])
    # Step 1: Create an adjacency list
    adjacency_list = {}
    for i_edge in range(edges.shape[0]):
        u, v = edges[i_edge,0],edges[i_edge,1]
        if u not in adjacency_list:
            adjacency_list[u] = []
        if v not in adjacency_list:
            adjacency_list[v] = []
        adjacency_list[u].append(v)
        adjacency_list[v].append(u)

    # Step 2: Choose a starting vertex
    start_vertex = list(adjacency_list.keys())[0]

    # There may be more than one closed path... (e.g., V2v V2d)
    set_unordered = set(unique_vx)
    ordered_list_multi = []
    set_ordered = set(sum(ordered_list_multi, [])) # flatten list and make it a set
    missing_vx = list(set_unordered - set_ordered)
    while len(missing_vx)!=0:
        start_vertex = missing_vx[0]
        ordered_list = dag_traverse_graph(start_vertex, adjacency_list)
        ordered_list_multi.append(ordered_list)
        set_ordered = set(sum(ordered_list_multi, [])) # flatten list and make it a set
        missing_vx = list(set_unordered - set_ordered)

    # OPTION: Convert ordered list to edge list
    # ordered_edges = [(ordered_list[i], ordered_list[(i + 1) % len(ordered_list)]) for i in range(len(ordered_list))]


    return ordered_list_multi#np.array(ordered_list)

def dag_traverse_graph(start_vertex, adjacency_list):
    '''Traverse a graph using 
    i.e. we have a list of edges, and we want to find the order of the vertices
    Useful for drawing an ROI

    start_vertex: int       - the starting vertex
    adjacency_list: dict    - the adjacency list of the graph   
                            - keys are vertices, values are the neighbours                             
    '''    
    # Build an ordered list of vertices, going through the graph
    ordered_list = []
    # Stack - to keep track of the vertices to visit
    stack = [start_vertex]
    # Visited - to keep track of the vertices we have visited
    visited = set()

    while stack:
        # Pop the last vertex from the stack
        current_vertex = stack.pop()
        # If we have not visited this vertex before...
        if current_vertex not in visited:
            # Mark as visited
            visited.add(current_vertex)
            # Add to ordered list
            ordered_list.append(current_vertex)
            # Visit neighbors in reverse order to maintain loop direction
            # (i.e., we want to go clockwise around the ROI)
            # this is added to the stack
            stack.extend(reversed(adjacency_list[current_vertex]))
            
    return ordered_list

svg_file = opj('path/to/sub/ctx/file', 'overlays.svg')


vx_list = []
for hemi in ['lh', 'rh']:
    if hemi == 'lh':
        hemi_roi_bool = v1_bool[:nvx[hemi]]
    else:
        hemi_roi_bool = v1_bool[nvx[hemi]:]
    
    border_vx_list, border_coords_list = dag_find_border_vx_in_order(
        hemi_roi_bool, mesh_info['inflated'][hemi], return_coords=True)

    # Loop through the border vertices (there may be 2 - one for each hemisphere)    
    # There maybe more if you have dorsal and ventral defined separately
    for ibvx,border_vx in enumerate(border_vx_list):
        this_roi_dict = {
            'hemi' : hemi,
            'roi' : 'V1',
            'border_vx' : border_vx,
            'border_coords' : border_coords_list[ibvx],
        }
        vx_list.append(this_roi_dict)      
    


Write everything to the svg using pycortex's tools

In [ ]:
from lxml import etree
from cortex import db
from cortex.svgoverlay import get_overlay, _make_layer, _find_layer, parser
mpts, mpolys = db.get_surf(sub, "flat", merge=True, nudge=True)
# Normalize coords as in pycortex
svgmpts = mpts[:,:2].copy()
svgmpts -= svgmpts.min(0)
svgmpts *= 1024 / svgmpts.max(0)[1] # Why 1024? I guess something about pixels?
svgmpts[:,1] = 1024 - svgmpts[:,1]
svgmpts_hemi = {}
svgmpts_hemi['lh'] = svgmpts[:nvx['lh'],:]
svgmpts_hemi['rh'] = svgmpts[nvx['lh']:,:]
new_vx_list = []
#
# NORMALIZE COORDS
for vx in vx_list:                        
    hemi = vx['hemi']
    vx['border_coords'][0] = svgmpts_hemi[hemi][vx['border_vx'],0].copy()
    vx['border_coords'][1] = svgmpts_hemi[hemi][vx['border_vx'],1].copy()
    vx['border_coords'][2] *= 0 # flat!       
    new_vx_list.append(vx)
vx_list = new_vx_list
# Now add to svg
svgroipack = get_overlay(sub, svg_file, mpts, mpolys)
# Add ROI boundaries
svg = etree.parse(svgroipack.svgfile, parser=parser)

for i,vx in enumerate(vx_list):
    roi_name = f"{vx['roi']}"
    roilayer = _make_layer(
        _find_layer(_find_layer(svg, "rois"),"shapes"),
        roi_name, 
        )
    x_coords = vx['border_coords'][0].copy()
    y_coords = vx['border_coords'][1].copy()
    # In svg language the path is defined by a string
    # M x1,y1 L x2,y2 L x3,y3 L x4,y4 Z 
    # where M is move to, L is line to, Z is close path
    path_data = f"M {x_coords[0]:.4f},{y_coords[0]:.4f} "            
    # Generate path data
    for i in range(1, len(x_coords)):
        path_data += f"L {x_coords[i]:.4f},{y_coords[i]:.4f} "
    # Close it at the end
    path_data += f"Z"            
    # Insert into SVG
    svgpath = etree.SubElement(roilayer, "path")
    stroke_col = '#ff0000' # red
    # Fill is none, stroke is red, stroke width is 1
    svgpath.attrib["style"] = f"fill:none;stroke:{stroke_col};stroke-width:1px;stroke-linecap:butt;stroke-linejoin:miter;stroke-opactiy:1"    
    # Add the path data
    svgpath.attrib["d"] = path_data
    # svgpath.attrib["sodipodi:nodetypes"] = "c" * len(pts)
    # break
# Save the svg
with open(svgroipack.svgfile, "wb") as xml:
    xml.write(etree.tostring(svg, pretty_print=True))


Now everything should be working and you have you're freesurfer ROIs in inkscape! 